In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
Nobs = 100 # nombre d'observations
Ncov = 5 # nombre de covariables
Pcov = 0.5 # probabilité qu'une covariable vaille 1

DataTemp = np.random.binomial(1,Pcov,(Nobs,Ncov))
Data = pd.DataFrame(DataTemp,columns=[f"X{i+1}" for i in range(Ncov)])

print(Data)


    X1  X2  X3  X4  X5
0    0   1   0   0   0
1    0   1   1   0   0
2    0   0   0   0   1
3    0   0   0   0   0
4    0   1   0   1   0
5    0   1   1   1   0
6    0   1   0   0   0
7    0   1   0   0   1
8    1   0   1   1   0
9    0   0   0   0   1
10   0   0   1   0   1
11   0   0   1   0   0
12   0   0   1   0   1
13   0   0   0   1   1
14   0   0   0   0   0
15   1   1   0   1   1
16   1   0   0   0   1
17   1   0   1   1   1
18   1   0   1   1   0
19   0   0   0   0   0
20   1   1   0   0   0
21   1   1   0   1   1
22   1   1   1   0   1
23   0   0   0   1   1
24   0   0   0   1   1
25   1   0   1   0   0
26   0   1   0   1   1
27   0   0   0   0   1
28   1   1   1   1   0
29   0   1   1   1   0
30   0   1   1   0   1
31   1   1   0   1   0
32   1   0   0   1   1
33   0   1   1   0   0
34   0   0   1   1   1
35   1   0   1   0   1
36   1   1   0   1   0
37   1   1   0   1   0
38   0   1   1   1   1
39   1   0   1   1   0
40   0   1   0   1   1
41   1   0   1   1   0
42   0   1 